In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import json
# Import the Class class.
from classy import Class
# Import velocileptors.
from velocileptors.LPT.lpt_rsd_fftw import LPT_RSD
# Gamma function for wp(R).
from scipy.special import gamma

## LAE clustering

Let's look at the clustering of LAEs in the mocks, using pre-generated data files created by AbacusHOD and theoretical predictions from Velocileptors.

In [ ]:
lae = json.load(open("lae_clustering_c000_ph100_s.json","r"))
Lbox= lae['BoxSize']
#
print("# OmM={:.3f}, H0={:.2f}, z={:.2f}".format(lae['Omega_M'],lae['H0'],lae['Redshift']))
print("# Simulation box of side length {:.0f}Mpc/h.".format(Lbox))
print("# {:33s} {:>6s} {:>10s}".format('HOD params','fsat','nbar'))
for samp in lae['mocks']:
    hodstr = ""
    for p in samp['hod']: hodstr += " {:6.2f}".format(p)
    print(hodstr+" {:6.2f} {:10.2e}".format(samp['fsat'],samp['nobj']/Lbox**3))

### Linear theory

First set up the linear theory and background quantities for this cosmology using CLASS.

In [ ]:
# Set up the class instance.
params = {
    'output': 'tCl lCl mPk',
    'l_max_scalars': 2000,
    'P_k_max_h/Mpc': 50.,
    'z_pk': '0.0,10',
    'lensing': 'yes',
    'A_s': np.exp(3.040)*1e-10,
    'n_s': 0.96824,
    'h': 0.6770,
    'N_ur': 2.0328,
    'N_ncdm': 1,
    'tau_reio': 0.0568,
    'omega_b': 0.022447,
    'omega_cdm': 0.11923}
# Now update some keys with N-body values.
for k in ['n_s','omega_b','omega_cdm',\
          'omega_ncdm','N_ncdm','N_ur']:
    params[k] = lae[k]
params['h'] = lae['H0']/100.0
#
cosmo = Class()
cosmo.set(params)
cosmo.compute()
#
wb = cosmo.omega_b()
wnu= params['omega_ncdm'] # wnu= 0.0106 * params['m_ncdm']
#
print("OmegaM=",cosmo.Omega_m())
print("sigma8=",cosmo.sigma8())
print("hubble=",cosmo.h())
print("omegab=",wb)
print("omegav=",wnu)
#
cosmo.get_current_derived_parameters(['H0','Omega_Lambda',\
                                      'age','conformal_age','Neff',\
                                      'z_reio','100*theta_s','rs_rec','rs_d'])

In [ ]:
zz = lae['Redshift']
ff = cosmo.scale_independent_growth_factor_f(zz)
print("z={:.2f}, f={:.4f}".format(zz,ff))
#
hub= cosmo.h()                # To convert to "conventional" Mpc/h units.
kk = np.logspace(-4.0,1.5,1000)
pk = np.array( [cosmo.pk(k*hub,zz)*hub**3 for k in kk] )
pl = np.array( [cosmo.pk_lin(k*hub,zz)*hub**3 for k in kk] )
#
# Compute the Zeldovich displacement and hence k_{nl}.
knl= 1/np.sqrt( np.trapz(pl,x=kk)/6./np.pi**2 )
print("knl=",knl," h/Mpc.")

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,4.5))
ax.plot(kk,pk,label="$z={:.1f}$".format(zz))
ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'$k\quad [h\,{\rm Mpc}^{-1}]$',fontsize=18)
ax.set_ylabel(r'$P(k)\quad [h^{-3}{\rm Mpc}^3]$',fontsize=18)

In [ ]:
def xi_mm(rr,zz):
    """Brute force xi_mm(r) for scalar r in Mpc/h."""
    kv = np.linspace(1e-3,50/rr,2500)
    ap = np.cos(np.pi/2*kv/kv[-1]) # Apodize the integral.
    j0 = np.sin(kv*rr)/(kv*rr)
    pv = np.array( [cosmo.pk(k*hub,zz)*hub**3 for k in kv] )
    xi = np.trapz(kv**2*pv/(2*np.pi**2) * j0 * ap,x=kv)
    return(xi)

In [ ]:
# Let's just look at "xi(r)" for the matter spectrum,
# using linear theory just to get a feel for sizes.
for rr in [1.0,2.0]:
    print("xi({:5.2f})={:5.2f}".format(rr,xi_mm(rr,zz))) 

### LAE data

Get some routines to help massage LAE data.

In [ ]:
# Schechter function fits from the literature.  The phi* definition
# is in terms of L, i.e. we need a ln(10) going to lgL [erg/s].
# phi* is in units of Mpc^{-3} and converted to Mpc/h below.
LFlist = []
LFlist.append({'zfid':2.2,'phis':3.37e-4,'Lstar':4.87e42,'alpha':-1.80,\
               'ref':'Konno+16'})
LFlist.append({'zfid':3.1,'phis':3.90e-4,'Lstar':8.49e42,'alpha':-1.80,\
               'ref':'Konno+16'})
LFlist.append({'zfid':3.7,'phis':3.31e-4,'Lstar':9.16e42,'alpha':-1.80,\
               'ref':'Konno+16'})
#LFlist.append({'zfid':3.7,'phis':3.4e-4,'Lstar':1.02e43,'alpha':-1.50,\
#               'ref':'Ouchi+08'})
LFlist.append({'zfid':5.7,'phis':4.44e-4,'Lstar':9.09e42,'alpha':-1.80,\
               'ref':'Konno+16'})

In [ ]:
def nbar_fit(lgL,zz):
    """Computes nbar, in [Mpc/h]^{-3}, given line luminosity lgL [erg/s]
    for LAEs."""
    # Choose the closest redshift in the table.
    delz= np.array([np.abs(zz-lf['zfid']) for lf in LFlist])
    lf  = LFlist[np.argmin(delz)]
    # Get the Schechter paramerers, converting to Mpc/h units.
    hub = cosmo.h()
    phis,lgLstar,alpha = lf['phis']/hub**3,np.log10(lf['Lstar']),lf['alpha']
    # Just brute-force the Schechter integral, it's plenty fast enough.
    logl = np.linspace(lgL,lgLstar+10,1000)
    ll   = 10.0**(logl-lgLstar)
    lf   = np.log(10)*phis * ll**(alpha+1) * np.exp(-ll)
    nbar = np.trapz(lf,logl)
    return(nbar)

In [ ]:
def flux2L(zz):
    """Convert a flux in erg/s/cm2 to log10(L) in [erg/s].
    Add the returned value to log10(flux) to get log10(L)."""
    Mpc_cm = 3.086e24 # 1 Mpc in cm.
    dL     = cosmo.luminosity_distance(zz)
    val    = np.log10(4*np.pi)+2*(np.log10(dL)+np.log10(Mpc_cm))
    return(val)

In [ ]:
# Some helpful numbers.
flux = 5e-17
lgL  = np.log10(flux) + flux2L(zz)
chi  = cosmo.comoving_distance(zz) * cosmo.h()
#
print("z    ={:5.1f}".format(zz))
print("Flux ={:10.2e} [erg/s/cm^2]".format(flux))
print("lgL  ={:6.2f} [erg/s]".format(lgL))
print("nbar ={:10.2e}".format(nbar_fit(lgL,zz))+r' [Mpc/h]^3')
print("chi  ={:5.0f}Mpc/h".format(chi))

In [ ]:
# How much would we need to downsample each HOD model to
# match the nbar for this flux limit?
ntar = nbar_fit(lgL,zz) * Lbox**3
print("# {:>4s} {:>10s}".format('lgMc','frac'))
for samp in lae['mocks']:
    print("{:6.2f} {:10.2e}".format(samp['hod'][0],ntar/samp['nobj']))

In [ ]:
# Table 2 of Khostovan+19: https://arxiv.org/pdf/1811.00556.pdf
# IA484: z=2.99 \pm 0.09
gam,r0,dr0 = 1.80,3.85,0.45 # Mpc/h.
# Compute wp(R) as:
R   = np.logspace(-1,1,25)
wpR = R**(1-gam) * r0**gam * np.sqrt(np.pi)*gamma(gam/2-0.5)/gamma(gam/2)

### Velocileptors

Set up the velocileptors instance and just compute an example with semi-randomly chosen parameters to exercise the code.

In [ ]:
lpt = LPT_RSD(kk,pl,kIR=0.2)

In [ ]:
# b1,b2,bs, b3: linear, quadratic & cubic bias parameters
# alpha0,alpha2,alpha4,alpha6: counterterms
# sn0,sn2,sn4: stochastic contributions to P(k) and sigma^2.
biases = [1.55,-0.5,0.1,0.0]
cterms = [-1.0,-2.0,0.0,0.0]
stoch  = [0, 0, 0]
pars   = biases + cterms + stoch
#
#
lpt.make_pltable(ff,apar=1,aperp=1,kmin=5e-3,kmax=1.0,nk=100,nmax=4)
xi0,xi2,xi4 = lpt.combine_bias_terms_xiell(pars)
#
fig,ax = plt.subplots(1,1,figsize=(6,4.5))
ax.plot(xi0[0], xi0[0]**2*xi0[1],label=r'$\ell = 0$')
ax.plot(xi2[0],-xi2[0]**2*xi2[1],label=r'$\ell = 2$')
ax.plot(xi4[0], xi4[0]**2*xi4[1],label=r'$\ell = 4$')
ax.legend(title='$z={:.1f}$'.format(zz))
ax.set_xlim(0.0,150.0)
ax.set_ylim(-10.,35.0)
ax.set_xscale('linear')
ax.set_yscale('linear')
ax.set_xlabel(r'$s\quad [h^{-1}\,{\rm Mpc}]$',fontsize=18)
ax.set_ylabel(r'$i^\ell\ s^2\xi_\ell(s)\quad [h^{-2}{\rm Mpc}^2]$',fontsize=18)

### Fit to N-body

Now we can try to fit to our N-body results.  For now we ignore variance reduction or other tricks.

In [ ]:
# Begin by looking at the wp(R) results, i.e. without
# redshift-space distortions.
fig,ax = plt.subplots(1,1,figsize=(6,4.5))
# First plot the N-body wp(R).
R,icol = np.array(lae['R']),0
for samp in lae['mocks']:
    ax.plot(R,samp['wp'],'s',color='C'+str(icol),\
            mfc='None',label="{:5.2f}".format(samp['hod'][0]))
    icol = (icol+1)%10
# Now plot a power-law fit to the data.  This fit is
# suspect because it was done in the data neglecting
# redshift-space distortions.
R   = np.logspace(0,1,25)
wpR = R**(1-gam) * r0**gam * np.sqrt(np.pi)*gamma(gam/2-0.5)/gamma(gam/2)
wpup= R**(1-gam) * (r0+dr0)**gam * np.sqrt(np.pi)*gamma(gam/2-0.5)/gamma(gam/2)
wpdn= R**(1-gam) * (r0-dr0)**gam * np.sqrt(np.pi)*gamma(gam/2-0.5)/gamma(gam/2)
ax.plot(R,wpR,'k-',label='IA484')
ax.fill_between(R,wpdn,wpup,color='lightgrey')
#
ax.legend(title='$z={:.1f}$'.format(zz))
ax.set_xlim(0.5, 50.0)
ax.set_ylim(1.0,150.0)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'$R\quad [h^{-1}\,{\rm Mpc}]$',fontsize=18)
ax.set_ylabel(r'$w_p(R)\quad [h^{-1}{\rm Mpc}]$',fontsize=18)
plt.tight_layout()
plt.savefig('lae_IA484_wp.png')

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,4.5))
# First plot the N-body xi_ell.
ss   = np.array(lae['R'])
s2   = ss**2
icol = 1
for samp in lae['mocks'][1:2]:
    ax.plot(ss, s2*samp['xi0'],'s-',color='C'+str(icol),\
            mfc='None',label="{:5.2f}".format(samp['hod'][0]))
    ax.plot(ss,-s2*samp['xi2'],'o',color='C'+str(icol),mfc='None')
    icol = (icol+1)%10
ax.plot(xi0[0], xi0[0]**2*xi0[1],'k--',label=r'$\ell = 0$')
ax.plot(xi2[0],-xi2[0]**2*xi2[1],'k:' ,label=r'$\ell = 2$')
ax.legend(title='$z={:.1f}$'.format(zz))
ax.set_xlim( 0,125.0)
ax.set_ylim(-5, 40.0)
ax.set_xscale('linear')
ax.set_yscale('linear')
ax.set_xlabel(r'$s\quad [h^{-1}\,{\rm Mpc}]$',fontsize=18)
ax.set_ylabel(r'$i^\ell\ s^2\xi_\ell(s)\quad [h^{-2}{\rm Mpc}^2]$',fontsize=18)
plt.tight_layout()
plt.savefig('lae_IA484_xi.png')

In [ ]:
# Now we can also look at the power spectra predicted by
# this model.  We may want to adjust the stochastic terms.
#
# b1,b2,bs, b3: linear, quadratic & cubic bias parameters
# alpha0,alpha2,alpha4,alpha6: counterterms
# sn0,sn2, sn4: stochastic contributions to P(k) and sigma^2.
stoch  = [0, 0, 0]
pars   = biases + cterms + stoch
#
kl,p0,p2,p4 = lpt.combine_bias_terms_pkell(pars)
#
fig,ax = plt.subplots(1,1,figsize=(6,4.5))
ax.plot(kl,kl*p0,'k--',label=r'$\ell = 0$')
ax.plot(kl,kl*p2,'k:' ,label=r'$\ell = 2$')
ax.legend(title='$z={:.1f}$'.format(zz))
ax.set_xlim(0.0,0.5)
ax.set_ylim(0.0,600)
ax.set_xscale('linear')
ax.set_yscale('linear')
ax.set_xlabel(r'$k\quad [h\,{\rm Mpc}^{-1}]$',fontsize=18)
ax.set_ylabel(r'$k P_\ell(k)\quad [h^{-2}{\rm Mpc}^2]$',fontsize=18)

### Sample surveys

Finally let's look at some sample surveys.

In [ ]:
lam = [4849.,5010.,6620.]
dlam= [ 229.,75.00,160.0]
#
area= 100 * (np.pi/180.)**2  # Survey area in sr.
#
print("# Wavelengths in Angstroms, distances in (comoving) Mpc/h.")
print("# Assume survey area of 100 sq.deg.")
print("# {:>3s} {:>5s} {:>5s} {:>6s} {:>6s} {:>6s} {:>10s}".\
      format("lam","dlam","zcen","dz","chi","dchi","dvol"))
for i in range(len(lam)):
    zmid,dz= lam[i] / 1216. - 1.,dlam[i]/ 1216.
    chi    = cosmo.comoving_distance(zmid) * cosmo.h()
    dchi   = dz / cosmo.Hubble(zmid)       * cosmo.h()
    dvol   = area * chi**2 * dchi
    outstr = "{:5.0f} {:5.0f}".format(lam[i],dlam[i])
    outstr+= " {:5.2f} {:6.3f}".format(zmid,dz)
    outstr+= " {:6.0f} {:6.0f}".format(chi,dchi)
    outstr+= " {:10.1e}".format(dvol)
    print(outstr)

# Done